# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset from Ironhack's database
pokemon = pd.read_csv('../data/pokemon.csv')

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here
pokemon.head(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False


In [6]:
(pokemon['Total'] == pokemon['HP'] + pokemon['Attack'] + pokemon['Defense'] + pokemon['Sp. Atk'] +\
pokemon['Sp. Def'] + pokemon['Speed']).sum()
# By the some of this bool we can confirm the hypothesis.

800

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [33]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

In [56]:
X = pokemon[['Type 1', 'Type 2']].fillna('0')

In [57]:
enc = OneHotEncoder(sparse = False)

In [58]:
onehot = enc.fit_transform(X)
onehot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [59]:
enc.get_feature_names()

array(['x0_Bug', 'x0_Dark', 'x0_Dragon', 'x0_Electric', 'x0_Fairy',
       'x0_Fighting', 'x0_Fire', 'x0_Flying', 'x0_Ghost', 'x0_Grass',
       'x0_Ground', 'x0_Ice', 'x0_Normal', 'x0_Poison', 'x0_Psychic',
       'x0_Rock', 'x0_Steel', 'x0_Water', 'x1_0', 'x1_Bug', 'x1_Dark',
       'x1_Dragon', 'x1_Electric', 'x1_Fairy', 'x1_Fighting', 'x1_Fire',
       'x1_Flying', 'x1_Ghost', 'x1_Grass', 'x1_Ground', 'x1_Ice',
       'x1_Normal', 'x1_Poison', 'x1_Psychic', 'x1_Rock', 'x1_Steel',
       'x1_Water'], dtype=object)

In [61]:
encoded = pd.DataFrame(onehot, columns = enc.get_feature_names())

## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [72]:
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [76]:
# your code here
encoded_pokemon = pd.concat([pokemon['Total'], encoded], axis = 1)
encoded_pokemon.corr().sort_values(by = 'Total', ascending = False)

,Total,x0_Bug,x0_Dark,x0_Dragon,x0_Electric,x0_Fairy,x0_Fighting,x0_Fire,x0_Flying,x0_Ghost,...,x1_Ghost,x1_Grass,x1_Ground,x1_Ice,x1_Normal,x1_Poison,x1_Psychic,x1_Rock,x1_Steel,x1_Water
Total,1.000000,-0.143957,0.017818,0.196532,0.016715,-0.026948,-0.029086,0.050527,0.029504,0.007594,...,-0.004885,-0.039224,0.016486,0.100870,-0.013956,-0.067837,0.076054,-0.000512,0.070307,-0.018800
x0_Dragon,0.196532,-0.062713,-0.040984,1.000000,-0.049245,-0.030077,-0.038149,-0.053820,-0.014470,-0.041667,...,-0.027243,-0.036662,0.112272,0.118700,-0.014470,-0.043005,0.085964,-0.027243,-0.034325,-0.027243
x1_Fighting,0.138726,-0.006089,0.036251,-0.037412,-0.044216,-0.027006,-0.034254,0.151836,-0.012992,-0.037412,...,-0.024461,-0.032918,-0.039203,-0.024461,-0.012992,-0.038614,-0.038017,-0.024461,-0.030820,-0.024461
x1_Dragon,0.115240,-0.046612,0.100556,-0.030969,0.000370,-0.022355,-0.028355,-0.005812,0.228243,0.055056,...,-0.020248,-0.027249,-0.032452,-0.020248,-0.010755,-0.031964,-0.031470,-0.020248,-0.025513,-0.020248
x1_Ice,0.100870,-0.041003,0.071991,0.118700,0.009617,-0.019665,-0.024943,-0.035189,-0.009461,-0.027243,...,-0.017812,-0.023970,-0.028547,1.000000,-0.009461,-0.028118,-0.027683,-0.017812,-0.022443,-0.017812
x0_Psychic,0.094364,-0.085096,-0.055611,-0.056538,-0.066820,-0.040812,-0.051765,-0.073029,-0.019634,-0.056538,...,0.000093,-0.021819,-0.059244,-0.036965,-0.019634,-0.058354,-0.057452,-0.036965,-0.046576,-0.036965
x0_Steel,0.082000,-0.057419,-0.037524,-0.038149,-0.045088,-0.027538,-0.034929,-0.049277,-0.013248,-0.038149,...,0.186213,-0.033567,0.027708,-0.024943,-0.013248,-0.039375,0.204881,0.133424,-0.031428,-0.024943
x1_Psychic,0.076054,-0.063727,0.023490,0.085964,-0.050041,-0.030563,0.065654,-0.003697,-0.014704,-0.042340,...,-0.027683,-0.037254,-0.044367,-0.027683,-0.014704,-0.043700,1.000000,-0.027683,-0.034880,-0.027683
x1_Fire,0.073234,0.035349,0.135074,0.027289,0.015337,-0.018183,-0.023063,-0.032537,-0.008748,0.132246,...,-0.016469,-0.022164,-0.026396,-0.016469,-0.008748,-0.025999,-0.025597,-0.016469,-0.020751,-0.016469
x1_Steel,0.070307,0.138928,0.045446,-0.034325,0.060014,-0.024778,0.053226,-0.013332,-0.011921,-0.034325,...,-0.022443,-0.030202,-0.035969,-0.022443,-0.011921,-0.035428,-0.034880,-0.022443,1.000000,-0.022443


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [81]:
# your code here
dragon = (encoded_pokemon['x0_Dragon'] == 1)
encoded_pokemon.loc[dragon,:].corr().sort_values(by = 'Total', ascending = False)

,Total,x0_Bug,x0_Dark,x0_Dragon,x0_Electric,x0_Fairy,x0_Fighting,x0_Fire,x0_Flying,x0_Ghost,...,x1_Ghost,x1_Grass,x1_Ground,x1_Ice,x1_Normal,x1_Poison,x1_Psychic,x1_Rock,x1_Steel,x1_Water
Total,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.085285,0.304144,NaN,NaN,0.261146,NaN,NaN,NaN
x1_Ice,0.304144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.138409,1.000000,NaN,NaN,-0.121566,NaN,NaN,NaN
x1_Flying,0.304104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.206725,-0.154508,NaN,NaN,-0.181568,NaN,NaN,NaN
x1_Psychic,0.261146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.162650,-0.121566,NaN,NaN,1.000000,NaN,NaN,NaN
x1_Electric,0.161521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.077290,-0.057767,NaN,NaN,-0.067884,NaN,NaN,NaN
x1_Fire,0.161521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.077290,-0.057767,NaN,NaN,-0.067884,NaN,NaN,NaN
x1_Fairy,0.049240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.077290,-0.057767,NaN,NaN,-0.067884,NaN,NaN,NaN
x1_Ground,-0.085285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.000000,-0.138409,NaN,NaN,-0.162650,NaN,NaN,NaN
x1_0,-0.689580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.311451,-0.232781,NaN,NaN,-0.273551,NaN,NaN,NaN
x0_Bug,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
